In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.6.0'

In [2]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [3]:
model = keras.models.load_model('xception_v1_09_0.803.h5')


In [4]:
img = load_img('./dataset/test_black.jpg', target_size=(150, 150))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [5]:
preds = model.predict(X)
preds

array([[ 13.855396  ,   3.202181  ,   5.1696057 ,  -3.5375361 ,
          6.2616096 ,  -2.232081  ,   2.1969457 ,  -8.189516  ,
         -7.0077405 ,  -5.077041  ,  -4.647565  ,  -9.379538  ,
         -2.9228396 ,  -7.043243  ,  -5.428063  , -11.299324  ,
        -10.534654  ,  -4.9850574 , -15.3460455 , -21.140926  ,
        -11.158437  ,  -6.910347  ,  -3.0318916 , -16.477018  ,
         -9.474036  , -10.201892  ,  -7.909956  ,  -6.8831253 ,
         -3.5060778 ,   0.17380346,  -7.3390613 ,  -6.37476   ,
         -2.6427524 , -12.875496  ,  -5.3285685 , -11.497002  ,
         -4.0317116 ]], dtype=float32)

In [6]:
classes = ['black_dress',
 'black_pants',
 'black_shirt',
 'black_shoes',
 'black_shorts',
 'black_suit',
 'blue_dress',
 'blue_pants',
 'blue_shirt',
 'blue_shoes',
 'blue_shorts',
 'brown_hoodie',
 'brown_pants',
 'brown_shoes',
 'green_pants',
 'green_shirt',
 'green_shoes',
 'green_shorts',
 'green_suit',
 'pink_hoodie',
 'pink_pants',
 'pink_skirt',
 'red_dress',
 'red_hoodie',
 'red_pants',
 'red_shirt',
 'red_shoes',
 'silver_shoes',
 'silver_skirt',
 'white_dress',
 'white_pants',
 'white_shoes',
 'white_shorts',
 'white_suit',
 'yellow_dress',
 'yellow_shorts',
 'yellow_skirt']

In [7]:
dict(zip(classes, preds[0]))

{'black_dress': 13.855396,
 'black_pants': 3.202181,
 'black_shirt': 5.1696057,
 'black_shoes': -3.5375361,
 'black_shorts': 6.2616096,
 'black_suit': -2.232081,
 'blue_dress': 2.1969457,
 'blue_pants': -8.189516,
 'blue_shirt': -7.0077405,
 'blue_shoes': -5.077041,
 'blue_shorts': -4.647565,
 'brown_hoodie': -9.379538,
 'brown_pants': -2.9228396,
 'brown_shoes': -7.043243,
 'green_pants': -5.428063,
 'green_shirt': -11.299324,
 'green_shoes': -10.534654,
 'green_shorts': -4.9850574,
 'green_suit': -15.3460455,
 'pink_hoodie': -21.140926,
 'pink_pants': -11.158437,
 'pink_skirt': -6.910347,
 'red_dress': -3.0318916,
 'red_hoodie': -16.477018,
 'red_pants': -9.474036,
 'red_shirt': -10.201892,
 'red_shoes': -7.909956,
 'silver_shoes': -6.8831253,
 'silver_skirt': -3.5060778,
 'white_dress': 0.17380346,
 'white_pants': -7.3390613,
 'white_shoes': -6.37476,
 'white_shorts': -2.6427524,
 'white_suit': -12.875496,
 'yellow_dress': -5.3285685,
 'yellow_shorts': -11.497002,
 'yellow_skirt': -

#### As you can see model predicted this as a black dress with highest probability which is correct!

#### Converting model to TFLite for deployment

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('xception-clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/hb/6ftrzv4d27l4qgry0h5tcc040000gn/T/tmpn1pgp5hs/assets


/Users/deepa/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [9]:
!pip install keras-image-helper

In [10]:
import tensorflow.lite as tflite

In [11]:
from keras_image_helper import create_preprocessor

In [12]:
interpreter = tflite.Interpreter(model_path='xception-clothing-model.tflite')
interpreter.allocate_tensors()
interpreter.get_input_details()

[{'name': 'input_26',
  'index': 0,
  'shape': array([  1, 150, 150,   3], dtype=int32),
  'shape_signature': array([ -1, 150, 150,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [13]:
interpreter.get_output_details()

[{'name': 'Identity',
  'index': 229,
  'shape': array([ 1, 37], dtype=int32),
  'shape_signature': array([-1, 37], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [14]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [15]:
preprocessor = create_preprocessor('xception', target_size=(150, 150))


In [ ]:
X = preprocessor.from_url(url)
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds